In [1]:
# little helpful tricks 
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [1]:
import pandas as pd
import csv
import cleaner

In [5]:
fd = 'all_workouts_fixed.csv'

In [6]:
all_workouts_fixed = cleaner.read_csv(fd)


# Filtering Logic

The user will be able to filter the information by four factors muscle group, body group, force, and equipment.

In [7]:
# a function to change all the strings in a column to lowercase
def to_lower(df: pd.DataFrame,column: str) -> None:
    df[column] = df[column].apply(lambda x: x.lower())

In [8]:
all_workouts_fixed

,name,body_group,force,muscle_group,equipment,description
0,Partner plank band row,Core,Hold,Abs,Band,The partner plank band row is an abdominal exe...
1,Banded crunch isometric hold,Core,Hold,Abs,Band,The banded crunch isometric hold is an exercis...
2,FYR Banded Plank Jack,Core,Hold,Abs,Band,The banded plank jack is a variation on the pl...
3,Banded crunch,Core,Hold,Abs,Band,The banded crunch is an exercise targeting the...
4,Crunch,Core,Hold,Abs,Band,The crunch is a popular core exercise targetin...
...,...,...,...,...,...,...
4533,Shoulder around the shoulder,Upper Body,Pull,Back,Bodyweight,"Starting posture: sitting or standing, the ba..."
4534,Kneeling and bend your knees and lift your legs,Lower Body,Push,Legs,Bodyweight,Starting posture: limbs to the ground. Step:...
4535,Turn around,Upper Body,Push,Shoulders,Bodyweight,"Starting posture: sitting or standing, the ba..."
4536,High -level flat support,Upper Body,Push,Chest,Bodyweight,Starting posture: Start high flat flat suppor...


In [10]:
force_categories = {
    'Cardio': 'N/A',
}

all_workouts_fixed = cleaner.create_column_based_on_dict(all_workouts_fixed, 'muscle_group', 'force', force_categories)


In [11]:
to_lower(all_workouts_fixed, 'muscle_group')
to_lower(all_workouts_fixed, 'body_group')
to_lower(all_workouts_fixed, 'force')
to_lower(all_workouts_fixed, 'equipment')


In [12]:
# A dictionary of the choices the user can make
user_options = {'Body Group': None, 'Force': None, 'Muscle Group': None, 'Equipment': None}

In [13]:
# Apply filtering based on user inputs
def user_data(df: pd.DataFrame, user_options: dict) -> pd.DataFrame:
    user_set = df.copy()
    for key, value in user_options.items():
        if value and value != 'none':
            column = key.replace(" ", "_").lower()
            if value != 'any':
                user_set = user_set[(user_set[column] == value) | (user_set[column].isin(['unknown', 'unavailable']))]
    return user_set

In [ ]:
# the main code to ask a user there preferences
print('Hello, this is the workout recommender, based on your input I will give you exercises.')

for item in user_options:
    repeat = True
    while repeat:
        try:
            answer = input(f'Input your selection for {item} or type list to see a list of what is available: ').lower()
            if answer == 'list':
                if item == "Body Group":
                    print(all_workouts_fixed['body_group'].unique())
                elif item == "Force":
                    if user_options["Body Group"]:
                       print(all_workouts_fixed[(all_workouts_fixed['body_group'] == user_options["Body Group"])]['force'].unique())
                elif item == "Muscle Group":
                    if user_options["Body Group"] and user_options["Force"]: # based off what you choose above should affect the out come of the list presented to the user
                       print(all_workouts_fixed[(all_workouts_fixed['body_group'] == user_options["Body Group"]) & (all_workouts_fixed['force'] == user_options["Force"])]['muscle_group'].unique())
                    elif user_options["Body Group"]:
                       print(all_workouts_fixed[all_workouts_fixed['body_group'] == user_options["Body Group"]]['muscle_group'].unique())
                    else:
                       print(all_workouts_fixed['muscle_group'].unique())
                elif item == "Equipment":
                    if user_options["Muscle Group"] and user_options["Body Group"]:
                        print(all_workouts_fixed[(all_workouts_fixed['muscle_group'] == user_options["Muscle Group"]) & (all_workouts_fixed['body_group'] == user_options["Body Group"])]['equipment'].unique())
                    elif user_options["Body Group"]:
                        print(all_workouts_fixed[all_workouts_fixed['body_group'] == user_options["Body Group"]]['equipment'].unique())
                    else:
                        print(all_workouts_fixed['equipment'].unique())
            elif answer in all_workouts_fixed[item.replace(" ", "_").lower()].unique() or answer in ['any', 'none']:
                print(f'Okay {answer} for {item}')
                user_options[item] = answer
                repeat = False
            else:
                print(f"Error: Invalid input for {item}. Please try again.\n")
        except ValueError:
            print("Error: Invalid input, please try again\n")

Hello, this is the workout recommender, based on your input I will give you exercises.
['core' 'lower body' 'upper body' 'cardio']
['core' 'lower body' 'upper body' 'cardio']
Okay upper body for Body Group
['pull' 'push' 'unknown' 'hold']
Okay pull for Force
['arms' 'forearms' 'back' 'neck' 'shoulders']
Okay forearms for Muscle Group
['band' 'barbell' 'kettlebell' 'dumbbell' 'other' 'cable' 'bodyweight'
 'plate' 'stretches' 'yoga']
Okay cable for Equipment


In [ ]:
# saves all the users data to a dataset for the user
users_exercises = user_data(all_workouts_fixed, user_options)
users_exercises.reset_index(drop=True, inplace=True)

In [ ]:
# Define sets, reps, and rest times based on goals
goal_details = {
    'strength': {'reps': '1-5', 'sets': '3-6', 'rest': '2-5 min', 'weight': 'Heavy'},
    'growth': {'reps': '6-12', 'sets': '3-6', 'rest': '60-90 sec', 'weight': 'Middle'},
    'endurance': {'reps': '12-20', 'sets': '2-3', 'rest': '30-60 sec', 'weight': 'Light'},
    'power': {'reps': '1-5 (Explosive)', 'sets': '3-5', 'rest': '2-5 min', 'weight': 'Heavy'},
    'weight loss': {'reps': '10', 'sets': '3', 'rest': '90 sec', 'weight': 'Middle-Light'},
    'general fitness': {'reps': '12-15', 'sets': '1-3', 'rest': '30-90 sec', 'weight': 'Middle-Light'}
}

In [ ]:
# a function based on the goal will display the proper set and rep ranges along with the rest in between sets
def u_goal_details(row, goal):
    sets = goal_details[goal]['sets']
    reps = goal_details[goal]['reps']
    rest = goal_details[goal]['rest']
    weight = goal_details[goal]['weight']
    row['goal'] = goal
    row['weight'] = weight
    row['sets'] = sets 
    row['reps'] = reps 
    row['rest'] = rest
    
    return row

In [ ]:
# prompts the user on there goal and how many days they can workout
goals = input(f'What is your goal? Muscle Growth, Strength, Weight loss, Power or Endurance? Choose one.').lower()
days = int(input(f'How many days can you workout?'))

In [ ]:
# appliss the function to the user dataset so thay have the additional information
users_exercises = users_exercises.apply(u_goal_details, goal=goals, axis=1)

In [ ]:
# Print filtered exercises with the reps, sets, rest time, goal, and how heavy weight they should use
print("Here are your recommended exercises:")
users_exercises.head()

Here are your recommended exercises:


,name,body_group,force,muscle_group,equipment,description,goal,weight,sets,reps,rest
0,Seated Two-Arm Palms-Up Low-Pulley Wrist Curl,upper body,pull,forearms,cable,not available,power,Heavy,3-5,1-5 (Explosive),2-5 min
1,Cable Wrist Curl,upper body,pull,forearms,cable,not available,power,Heavy,3-5,1-5 (Explosive),2-5 min
2,Single-arm cable push-down,upper body,pull,forearms,cable,The single-arm cable push-down is a single-joi...,power,Heavy,3-5,1-5 (Explosive),2-5 min
3,Cable Wrist Curl,upper body,pull,forearms,cable,Use a handle attachment. The cable should be s...,power,Heavy,3-5,1-5 (Explosive),2-5 min
4,Cable Wrist Extension,upper body,pull,forearms,cable,Use a handle attachment. The cable should be s...,power,Heavy,3-5,1-5 (Explosive),2-5 min


In [ ]:
# Fill NaN values in a column with a specified value.
def fill_nan(df: pd.DataFrame, column:str, value:any)-> pd.DataFrame:
    df[column].fillna(value, inplace=True)
    return df

# Apply cleaning functions
def apply_cleaning(df):
    force_categories = {'Cardio': 'N/A'}
    df = to_lower(df, 'muscle_group')
    df = to_lower(df, 'body_group')
    df = to_lower(df, 'force')
    df = to_lower(df, 'equipment')
    df = fill_nan(df, 'force', 'N/A')
    return df